# Project 3 - Transfer Learning

### team members:
### 1. Sasi Kanduri
### 2. Vikas Mishra
### 3. Ashish Thranath Kotian

### import libraries

In [496]:
import numpy as np
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model, image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D, Rescaling
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate

import tensorflow as tf

### Importing training/validation/test Data from the files downlaoded with image_downloader.py

In [497]:
df_train = pd.read_csv('final_df_train.csv', index_col=False)
df_validate = pd.read_csv('final_df_validate.csv', index_col=False)
df_test = pd.read_csv('final_df_test.csv', index_col=False)

y_train = df_train['2_way_label'].to_numpy()
y_validate = df_validate['2_way_label'].to_numpy()
y_test = df_test['2_way_label'].to_numpy()

print(f"train set x shape : {df_train.shape} y shape: {y_train.shape}")
print(f"validate set x shape : {df_validate.shape} y shape: {y_validate.shape}")
print(f"test set x shape : {df_test.shape} y shape: {y_test.shape}")


train set x shape : (4812, 2) y shape: (4812,)
validate set x shape : (968, 2) y shape: (968,)
test set x shape : (963, 2) y shape: (963,)


## Text Model with GLoVe Embedding for text data

### Contractions

In [498]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

### Function to clean the text using the contractions

In [499]:
def get_clean_text(x):
    if type(x) is str:
        x = x.lower()
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x


### Applying Contraction Function on Train/Validation/Testing Data

In [500]:
df_train['text with images'] = df_train['text with images'].apply(lambda x: get_clean_text(x))
df_validate['text with images'] = df_validate['text with images'].apply(lambda x: get_clean_text(x))
df_test['text with images'] = df_test['text with images'].apply(lambda x: get_clean_text(x))

### Train Data After Applying Contraction Function

In [501]:
df_train['text with images']

0       my walgreens offbrand mucinex was engraved wit...
1           hackers leak emails from uae ambassador to us
2                                puppy taking in the view
3       bride and groom exchange vows after fatal shoo...
4                                           major thermos
                              ...                        
4807    old lady cupboard smiles at you over the rims ...
4808          video how to stay motivated during workdays
4809                             the scourge of carpathia
4810    las vegas shooting multiple casualties near ma...
4811    a year quest to find the brother he never knew...
Name: text with images, Length: 4812, dtype: object

### Validation Data After Applying Contraction Function

In [502]:
df_validate['text with images']

0                             my xbox controller says hi
1                         new image from the mandalorian
2                          say hello to my little friend
3                             watch your step little one
4                this tree i found with a solo cup on it
                             ...                        
963    a rare picture of walter cronkite reacting to ...
964    i cant win till the dank memes have been beate...
965                               heres a two headed dog
966    victim of the pirate francois lolonnais having...
967                                       available here
Name: text with images, Length: 968, dtype: object

### Test Data After Applying Contraction Function

In [503]:
df_test['text with images']

0                                              stargazer
1                                                   yeah
2      pd phoenix car thief gets instructions from yo...
3      as trump accuses iran he has one problem his o...
4                                    believers hezbollah
                             ...                        
958                       buy domestic japan advertising
959          a spider crawled into my taillight and died
960        no buyers for hell michigan despite fire sale
961                                     he had it coming
962       im on a steve buscemi kick for whatever reason
Name: text with images, Length: 963, dtype: object

### Generating Tokens using keras tokenizer

In [504]:
text_train = df_train['text with images'].tolist()
text_validate = df_validate['text with images'].tolist()
text_test = df_test['text with images'].tolist()

token = Tokenizer()

token.fit_on_texts(text_train)


### vocabulary size

In [505]:
vocab_size  = len(token.word_index) + 1
vocab_size

9246

### Encoding Text to Sequences for Train/Validation/Test Data

In [506]:
encoded_text_train = token.texts_to_sequences(text_train)
encoded_text_validate = token.texts_to_sequences(text_validate)
encoded_text_test = token.texts_to_sequences(text_test)

### Padding

In [507]:
max_length = 120
text_train = pad_sequences(encoded_text_train, maxlen=max_length, padding='post')
text_validate = pad_sequences(encoded_text_validate, maxlen=max_length, padding='post')
text_test = pad_sequences(encoded_text_test, maxlen=max_length, padding='post')

### Converting data to numpy arrays

In [508]:
text_train = np.array(text_train)
text_validate = np.array(text_validate)
text_test = np.array(text_test)

### Reading the Glove Vectors for each word from the pretrained embeddings file 'glove.twitter.27B.25d.txt' - 25 dimensions

In [509]:
glove_vectors = dict()

In [510]:
%%time

file = open('glove.twitter.27B.50d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1: ])
    glove_vectors[word] = vectors
file.close()

CPU times: user 8.67 s, sys: 271 ms, total: 8.94 s
Wall time: 8.94 s


In [511]:
len(glove_vectors)

1193514

### Creating Word Matrix For Each Word In Token

In [512]:
word_vector_matrix = np.zeros((vocab_size, 50))

tokens = []
labels = []

for word, index in token.word_index.items():   # index returned here starts with 1 so we need set vocab_size = len(token.word_index) + 1  to be able to index up to the greatest token ID
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector


### Word matrix size

In [513]:
word_vector_matrix.shape

(9246, 50)

### Building The Glove Embedding Model

In [514]:
# text model
input1 = Input(shape = (max_length))
embedding = Embedding(vocab_size, 50, weights = [word_vector_matrix], trainable = False, name='embedding')(input1)

layer1 = Conv1D(64, 8, activation = 'relu')(embedding)
layer2 = MaxPooling1D(2)(layer1)
layer3 = Conv1D(32, 4, activation = 'relu')(layer2)
layer4 = MaxPooling1D(2)(layer3)

flat1 = Flatten()(layer4)
dense1 = Dense(128, activation='relu')(flat1)

## CNN model for images

### Converting target label to list for labels

In [515]:

y_labels_train = y_train.tolist()
y_labels_validate = y_validate.tolist()
y_labels_test = y_test.tolist()

### Creating dataset for our train/validation/test Data

In [516]:

image_train = image_dataset_from_directory("images_train", labels=y_labels_train, label_mode="binary", image_size=(64,64), batch_size=32, color_mode='rgb')
image_val = image_dataset_from_directory("images_validate", labels=y_labels_validate, label_mode="binary", image_size=(64,64), batch_size=32, color_mode='rgb')
image_test = image_dataset_from_directory("images_test", labels=y_labels_test, label_mode="binary", image_size=(64,64), batch_size=32, color_mode='rgb')

Found 4812 files belonging to 2 classes.
Found 968 files belonging to 2 classes.
Found 963 files belonging to 2 classes.


### Autotune with keras for performance

In [517]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = image_train.prefetch(buffer_size=AUTOTUNE)

# train data
data_batches = []

for batch in train_dataset:
    data_batches.append(batch[0])

image_data_train = np.concatenate(data_batches, axis=0)

#validation data
val_dataset = image_val.prefetch(buffer_size=AUTOTUNE)

data_batches = []

for batch in val_dataset:
    try:
        data_batches.append(batch[0])
    except:
        print(batch)

image_data_validate = np.concatenate(data_batches, axis=0)

# testd data
test_dataset = image_test.prefetch(buffer_size=AUTOTUNE)

data_batches = []

for batch in test_dataset:
    data_batches.append(batch[0])

image_data_test = np.concatenate(data_batches, axis=0)

2023-10-29 23:44:45.461120: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


In [518]:
print(image_data_train.shape)
print(image_data_validate.shape)
print(image_data_test.shape)

(4812, 64, 64, 3)
(968, 64, 64, 3)
(963, 64, 64, 3)


### Building the CNN Model

In [519]:
input2 = Input(shape=(64,64,3))
rescaling = Rescaling(1./255)(input2) # scale pixels

In [520]:
conv1 = Conv2D(64, (3, 3), activation='relu')(rescaling)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1)
conv2 = Conv2D(32, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2), padding='same')(conv2)
flat2 = Flatten()(pool2)
d_layer = Dense(128, activation='relu')(flat2)
dense2 = Dense(32, activation='relu')(d_layer)

## Creating a multimodal architecture by merging the dense layers of text and image models

In [521]:
merge = concatenate([dense1, dense2])

hidden1 = Dense(128, activation='relu')(merge)
hidden2 = Dense(32, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=[input1, input2], outputs=output)

model.summary()

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_84 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 input_83 (InputLayer)       [(None, 120)]                0         []                            
                                                                                                  
 rescaling_18 (Rescaling)    (None, 64, 64, 3)            0         ['input_84[0][0]']            
                                                                                                  
 embedding (Embedding)       (None, 120, 50)              462300    ['input_83[0][0]']            
                                                                                           

### Defining ModelCheckPoint & EarlyStopping

In [522]:

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpointer = ModelCheckpoint(filepath="best_nultimodal.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor, checkpointer], batch_size = 32,
            validation_data=([text_validate, image_data_validate], y_validate))

Epoch 1/50
151/151 [==============================] - ETA: 0s - loss: 0.5900 - accuracy: 0.6937
Epoch 1: val_loss improved from inf to 0.59165, saving model to best_nultimodal.hdf5
151/151 [==============================] - 20s 91ms/step - loss: 0.5900 - accuracy: 0.6937 - val_loss: 0.5917 - val_accuracy: 0.6818
Epoch 2/50
  4/151 [..............................] - ETA: 2s - loss: 0.5107 - accuracy: 0.7422

/Users/shashi/Downloads/testing/testenv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


151/151 [==============================] - ETA: 0s - loss: 0.5918 - accuracy: 0.7066
Epoch 2: val_loss did not improve from 0.59165
151/151 [==============================] - 4s 26ms/step - loss: 0.5918 - accuracy: 0.7066 - val_loss: 0.6140 - val_accuracy: 0.6911
Epoch 3/50
151/151 [==============================] - ETA: 0s - loss: 0.5524 - accuracy: 0.7294
Epoch 3: val_loss did not improve from 0.59165
151/151 [==============================] - 3s 23ms/step - loss: 0.5524 - accuracy: 0.7294 - val_loss: 0.7465 - val_accuracy: 0.6539
Epoch 4/50
151/151 [==============================] - ETA: 0s - loss: 0.8774 - accuracy: 0.6964
Epoch 4: val_loss did not improve from 0.59165
151/151 [==============================] - 4s 24ms/step - loss: 0.8774 - accuracy: 0.6964 - val_loss: 0.9779 - val_accuracy: 0.5992
Epoch 5/50
151/151 [==============================] - ETA: 0s - loss: 1.0231 - accuracy: 0.6629
Epoch 5: val_loss did not improve from 0.59165
151/151 [==============================] - 

### Predicting with test data

In [523]:
model.load_weights("best_nultimodal.hdf5")
pred = model.predict([text_test, image_data_test])

pred = (pred > 0.5)

31/31 [==============================] - 3s 43ms/step


### Results

In [524]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.75      0.81      0.78       571
           1       0.69      0.61      0.65       392

    accuracy                           0.73       963
   macro avg       0.72      0.71      0.71       963
weighted avg       0.73      0.73      0.73       963



## Additional Feature - VGG16

In [525]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Loading VGG16 model

input_vgg = Input(shape=(64,64,3))
rescaling = Rescaling(1./255)(input_vgg) # scale pixels

base_model = VGG16(weights="imagenet", include_top=False, input_tensor=rescaling)
base_model.trainable = False

base_model.summary()
last_layer_vgg = base_model.layers[-1].output

flatten_layer = Flatten()(last_layer_vgg)
dense2 = Dense(128, activation='relu')(flatten_layer)

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_85 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 rescaling_19 (Rescaling)    (None, 64, 64, 3)         0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584

In [526]:
merge = concatenate([dense1, dense2])

hidden1 = Dense(128, activation='relu')(merge)
hidden2 = Dense(32, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)


model = Model(inputs=[input1, input_vgg], outputs=output)

model.summary()

Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_85 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 rescaling_19 (Rescaling)    (None, 64, 64, 3)            0         ['input_85[0][0]']            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 64, 64, 64)           1792      ['rescaling_19[0][0]']        
                                                                                                  
 block1_conv2 (Conv2D)       (None, 64, 64, 64)           36928     ['block1_conv1[0][0]']        
                                                                                           

Total params: 15616829 (59.57 MB)
Trainable params: 439841 (1.68 MB)
Non-trainable params: 15176988 (57.90 MB)
__________________________________________________________________________________________________


In [527]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# checkpointer = ModelCheckpoint(filepath="best_nultimodal_with_vgg.hdf5", verbose=2, save_best_only=True, monitor='val_loss')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

model.fit([text_train, image_data_train], y_train, epochs=50, callbacks=[monitor], batch_size = 32,
            validation_data=([text_validate, image_data_validate], y_validate))

Epoch 1/50
151/151 [==============================] - ETA: 0s - loss: 0.5882 - accuracy: 0.7105
Epoch 1: val_loss improved from inf to 0.58459, saving model to best_nultimodal_with_vgg.hdf5
151/151 [==============================] - 23s 108ms/step - loss: 0.5882 - accuracy: 0.7105 - val_loss: 0.5846 - val_accuracy: 0.6994
Epoch 2/50
  3/151 [..............................] - ETA: 4s - loss: 0.4767 - accuracy: 0.7188

/Users/shashi/Downloads/testing/testenv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


151/151 [==============================] - ETA: 0s - loss: 0.4978 - accuracy: 0.7635
Epoch 2: val_loss did not improve from 0.58459
151/151 [==============================] - 5s 33ms/step - loss: 0.4978 - accuracy: 0.7635 - val_loss: 0.6071 - val_accuracy: 0.6860
Epoch 3/50
151/151 [==============================] - ETA: 0s - loss: 0.4285 - accuracy: 0.8078
Epoch 3: val_loss did not improve from 0.58459
151/151 [==============================] - 5s 32ms/step - loss: 0.4285 - accuracy: 0.8078 - val_loss: 0.6894 - val_accuracy: 0.7004
Epoch 4/50
151/151 [==============================] - ETA: 0s - loss: 0.3074 - accuracy: 0.8705
Epoch 4: val_loss did not improve from 0.58459
151/151 [==============================] - 5s 32ms/step - loss: 0.3074 - accuracy: 0.8705 - val_loss: 0.8482 - val_accuracy: 0.6787
Epoch 5/50
149/151 [============================>.] - ETA: 0s - loss: 0.2085 - accuracy: 0.9197
Epoch 5: val_loss did not improve from 0.58459
151/151 [==============================] - 

In [528]:
# model.load_weights("best_nultimodal_with_vgg.hdf5")
pred = model.predict([text_test, image_data_test])
pred = (pred > 0.5)

31/31 [==============================] - 4s 89ms/step


### Results

In [529]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.76      0.79      0.77       571
           1       0.67      0.63      0.65       392

    accuracy                           0.72       963
   macro avg       0.71      0.71      0.71       963
weighted avg       0.72      0.72      0.72       963

